# Compare Pubmed Central Corpus with bioRxiv Corpus

In [1]:
%load_ext autoreload
%autoreload 2

from collections import Counter
import csv
from pathlib import Path
import pickle
import string

import pandas as pd
import spacy
from tqdm import tqdm

from annorxiver_modules.document_helper import dump_article_text

In [2]:
lemma_model = spacy.load("en_core_web_sm")
lemma_model.max_length = 9000000 

# Calculate Word Frequency of bioRxiv

In [3]:
biorxiv_map_df = (
    pd.read_csv("../exploratory_data_analysis/output/biorxiv_article_metadata.tsv", sep="\t")
    .groupby("doi")
    .agg({"document":"first", "doi":"last"})
)
print(biorxiv_map_df.shape)
biorxiv_map_df.head()

(71118, 2)


,document,doi
doi,,
10.1101/000026,000026_v1.xml,10.1101/000026
10.1101/000042,000042_v1.xml,10.1101/000042
10.1101/000067,000067_v1.xml,10.1101/000067
10.1101/000091,000091_v1.xml,10.1101/000091
10.1101/000109,000109_v1.xml,10.1101/000109


In [4]:
Path("output/biorxiv_word_counts/").mkdir(parents=True, exist_ok=True)

In [5]:
sentence_length = {}
for document in tqdm(biorxiv_map_df.document.tolist()):
    
    document_text = dump_article_text(
        file_path=f"../biorxiv_articles/{document}",
        xpath_str="//abstract/p|//abstract/title|//body/sec//p|//body/sec//title",
        remove_stop_words=False
    )

    doc = lemma_model(
        " ".join(document_text),  
        disable = ['ner']
    )
    
    tokens = [
        (str(tok).lower(), tok.pos_, tok.dep_) 
        for tok in doc 
        if tok.text.lower() not in string.punctuation
    ]
    
    sentence_length[document] = [len(sent) for sent in doc.sents]
    
    with open(f"output/biorxiv_word_counts/{document}.tsv", "w") as file:
        writer = csv.DictWriter(
            file, fieldnames=["lemma", "pos_tag", "dep_tag", "count"],
            delimiter="\t"
        )
        
        writer.writeheader()
    
        lemma_stats = Counter(tokens)          
        writer.writerows([
            {
                "lemma":val[0][0],
                "pos_tag":val[0][1],
                "dep_tag":val[0][2],
                "count":val[1]
            }
            for val in lemma_stats.items()
        ])        

100%|██████████| 71118/71118 [9:26:59<00:00,  2.09it/s]   


In [6]:
pickle.dump(
    sentence_length, 
    open("output/biorxiv_sentence_length.pkl", "wb")
)

# Calculate Word Frequency of Pubmed Central

In [3]:
pmc_map_df = (
    pd.read_csv(
        "../../pmc/exploratory_data_analysis/output/pubmed_central_journal_paper_map.tsv.xz", 
        sep="\t"
    )
    .query("article_type=='research-article'")
)
print(pmc_map_df.shape)
pmc_map_df.head()

(1977651, 4)


,journal,article_type,doi,pmcid
0,Environ_Health,research-article,10.1186/1476-069X-5-22,PMC1552054
1,Environ_Health,research-article,10.1186/1476-069X-4-12,PMC1226148
3,Environ_Health,research-article,10.1186/s12940-017-0316-3,PMC5635510
4,Environ_Health,research-article,10.1186/1476-069X-10-46,PMC3125232
5,Environ_Health,research-article,10.1186/1476-069X-11-91,PMC3533997


In [4]:
Path("../../pmc/pmc_corpus/pmc_word_counts/").mkdir(parents=True, exist_ok=True)

In [ ]:
sentence_length = {}
for document in tqdm(pmc_map_df[["journal", "pmcid"]].values.tolist()):
    
    #Skip files that dont exist or files already parsed
    if (
        not Path(f"../../pmc/journals/{document[0]}/{document[1]}.nxml").exists()
        or Path(f"../../pmc/pmc_corpus/pmc_word_counts/{document[1]}.tsv").exists()
    ):
        continue
    
    document_text = dump_article_text(
        file_path=f"../../pmc/journals/{document[0]}/{document[1]}.nxml",
        xpath_str="//abstract/sec/*|//body/sec/*",
        remove_stop_words=False
    )
    
    doc = lemma_model(
        " ".join(document_text),  
        disable = ['ner']
    )
    
    tokens = [
        (str(tok).lower(), tok.pos_, tok.dep_) 
        for tok in doc 
        if tok.text.lower() not in string.punctuation
    ]
    
    sentence_length[document[1]] = [len(sent) for sent in doc.sents]
    
    with open(f"../../pmc/pmc_corpus/pmc_word_counts/{document[1]}.tsv", "w") as file:
        writer = csv.DictWriter(
            file, fieldnames=["lemma", "pos_tag", "dep_tag", "count"],
            delimiter="\t"
        )
        
        writer.writeheader()
    
        lemma_stats = Counter(tokens)          
        writer.writerows([
            {
                "lemma":val[0][0],
                "pos_tag":val[0][1],
                "dep_tag":val[0][2],
                "count":val[1]
            }
            for val in lemma_stats.items()
        ])

 69%|██████▉   | 1362827/1977651 [58:02:12<55:23:48,  3.08it/s]  

In [ ]:
pickle.dump(
    sentence_length, 
    open("../../pmc/pmc_corpus/pmc_sentence_length.pkl", "wb")
)